In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from nn_analysis import load_model_for_inference
from dataloader import get_dataloader

import matplotlib.pyplot as plt

# Lowest-loss images?

Which images in the dataset have the lowest loss?

Investigation using [efficient-donkey-47](https://wandb.ai/ajacobsen/ulc-malaria-autofocus/runs/3at65ww4?workspace=user-ajacobsen)

In [7]:
net = load_model_for_inference("trained_models/efficient-donkey-final.pth")
dataloader = get_dataloader("training_data", 128)

FileNotFoundError: [Errno 2] No such file or directory: '-f'